## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,46.0,87,0,11.70,clear sky
1,1,Kapaa,US,22.0752,-159.3190,75.0,100,90,11.50,mist
2,2,Mataura,NZ,-46.1927,168.8643,61.0,76,1,1.01,clear sky
3,3,Ushuaia,AR,-54.8000,-68.3000,51.8,71,75,31.07,light rain
4,4,Tautira,PF,-17.7333,-149.1500,87.8,66,40,8.05,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
isNum = False
while not isNum:
    try:
        min_temp = float(input("Enter the minimum temperature you're comfortable with: "))
        isNum = True
    except ValueError:
        print("That amount isn't a number. Please input a valid number.")
isNum = False
while not isNum:
    try:
        max_temp = float(input("Enter the maximum temperature you're comfortable with: "))
        isNum = True
    except ValueError:
        print("That amount isn't a number. Please input a valid number.")

Enter the minimum temperature you're comfortable with: 10
Enter the maximum temperature you're comfortable with: 55


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
comfy_cities_df = city_data_df.loc[((city_data_df["Max Temp"] > min_temp) & (city_data_df["Max Temp"] < max_temp))]
comfy_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,46.00,87,0,11.70,clear sky
3,3,Ushuaia,AR,-54.8000,-68.3000,51.80,71,75,31.07,light rain
5,5,Sarkand,KZ,45.4100,79.9186,41.09,52,100,5.93,overcast clouds
6,6,Pustoshka,RU,56.3355,29.3690,34.27,97,100,11.83,overcast clouds
9,9,Hami,CN,42.8000,93.4500,47.28,35,99,5.93,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
comfy_cities_df.count()

City_ID                225
City                   225
Country                225
Lat                    225
Lng                    225
Max Temp               225
Humidity               225
Cloudiness             225
Wind Speed             225
Current Description    225
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = comfy_cities_df.dropna()
clean_df.count()

City_ID                225
City                   225
Country                225
Lat                    225
Lng                    225
Max Temp               225
Humidity               225
Cloudiness             225
Wind Speed             225
Current Description    225
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,46.00,clear sky,-33.6500,115.3333,
3,Ushuaia,AR,51.80,light rain,-54.8000,-68.3000,
5,Sarkand,KZ,41.09,overcast clouds,45.4100,79.9186,
6,Pustoshka,RU,34.27,overcast clouds,56.3355,29.3690,
9,Hami,CN,47.28,overcast clouds,42.8000,93.4500,
10,Vila,PT,39.99,few clouds,42.0304,-8.1588,
12,Vostok,RU,22.77,clear sky,46.4856,135.8833,
24,Nikolskoye,RU,32.00,snow,59.7035,30.7861,
30,Tasiilaq,GL,15.80,scattered clouds,65.6145,-37.6368,
35,Belyy Yar,RU,24.80,snow,53.6039,91.3903,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!=""]
clean_hotel_df.count()

City                   192
Country                192
Max Temp               192
Current Description    192
Lat                    192
Lng                    192
Hotel Name             192
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))